# Iteration 6.1

Reduce LR on Plateau

In [ ]:
data_dir = "../../Merge Set Ordered"
iteration = "iteration_6_1"
model_dir = f'../../models/best_model_{iteration}.h5'

In [ ]:
from keras.applications.mobilenet_v3 import preprocess_input


# Now import HomemadeDataloader from homemade_dataloader.py
from homemade_dataloader import DataloaderFactory


batch_size=32
dist = [1171,1171,1171,1171,1171,1171,1171]
print(sum(dist))
histories = []

factory = DataloaderFactory(data_dir, batch_size=batch_size, image_size=(224,224), set_distribution=(70,15,15), class_distribution=dist, preprocess_function=preprocess_input)

train_generator, validation_generator, test_generator = factory.get_dataloaders()

In [ ]:
# Example AUC scores for each class (from the PR curve)
auc_scores = [0.51, 0.96, 0.81, 0.55, 0.64, 0.66, 0.92]  # Replace with your actual AUC values

# Calculate weights inversely proportional to AUC scores
class_weights = {i: 1 / score for i, score in enumerate(auc_scores)}

# Normalize the weights to keep them reasonably scaled
max_weight = max(class_weights.values())
class_weights = {k: v / max_weight for k, v in class_weights.items()}

weights = class_weights

print("Class Weights:", class_weights)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
from custom_model_iteration_6 import CustomModel

model = CustomModel(number_of_samples=train_generator.samples)
model.compile(optimizer="SGD")

In [ ]:
history1 = model.fit_epochs(train_generator, validation_generator, epochs=5, class_weight=weights, checkpoint_path=model_dir)
histories.append(history1)

In [ ]:
model.unfreeze()
model.lr_find(train_generator, validation_generator)

In [ ]:
history2 = model.fit_epochs(train_generator, validation_generator, epochs=50, checkpoint_path=model_dir, lr=[1e-4, 1e-3])
histories.append(history2)

In [ ]:
from tensorflow.keras.models import load_model

m = load_model(model_dir)
show_all_plots(histories, m, test_generator)

# Iteration 6.2

Adam

In [ ]:
data_dir = "../../Merge Set Ordered"
iteration = "iteration_6_2"
model_dir = f'../../models/best_model_{iteration}.h5'

In [ ]:
from keras.applications.mobilenet_v3 import preprocess_input


# Now import HomemadeDataloader from homemade_dataloader.py
from homemade_dataloader import DataloaderFactory


batch_size=32
dist = [1171,1171,1171,1171,1171,1171,1171]
print(sum(dist))
histories = []

factory = DataloaderFactory(data_dir, batch_size=batch_size, image_size=(224,224), set_distribution=(70,15,15), class_distribution=dist, preprocess_function=preprocess_input)

train_generator, validation_generator, test_generator = factory.get_dataloaders() 

In [ ]:
# Example AUC scores for each class (from the PR curve)
auc_scores = [0.51, 0.96, 0.81, 0.55, 0.64, 0.66, 0.92]  # Replace with your actual AUC values

# Calculate weights inversely proportional to AUC scores
class_weights = {i: 1 / score for i, score in enumerate(auc_scores)}

# Normalize the weights to keep them reasonably scaled
max_weight = max(class_weights.values())
class_weights = {k: v / max_weight for k, v in class_weights.items()}

weights = class_weights

print("Class Weights:", class_weights)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
from custom_model_iteration_6 import CustomModel

model = CustomModel(number_of_samples=train_generator.samples)
model.compile(optimizer="ADAM")

In [ ]:
history1 = model.fit_epochs(train_generator, validation_generator, epochs=5, class_weight=weights, checkpoint_path=model_dir)
histories.append(history1)

In [ ]:
model.unfreeze()
model.lr_find(train_generator, validation_generator)

In [ ]:
history2 = model.fit_epochs(train_generator, validation_generator, epochs=50, checkpoint_path=model_dir, lr=[1e-4, 1e-3])
histories.append(history2)

In [ ]:
from tensorflow.keras.models import load_model

m = load_model(model_dir)
show_all_plots(histories, m, test_generator)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_curve, average_precision_score, f1_score
from sklearn.preprocessing import label_binarize

class CombinedHistory:
    def __init__(self, combined_history_dict):
        self.history = combined_history_dict

def combine_histories(histories):
    combined_history = {}
    for key in histories[0].history.keys():
        combined_history[key] = []
        for history in histories:
            combined_history[key].extend(history.history[key])
    return CombinedHistory(combined_history)

# Define the function to plot all metrics including balanced accuracy
def show_all_plots(histories, model, validation_generator):


    if isinstance(histories, list):  # Check if it's a list of histories
        history = combine_histories(histories)
    else:  # If single history is passed, use it directly
        history = histories.history
    


    # Assuming `validation_generator` has class indices mapped to class names
    class_names = list(validation_generator.class_names)
    print(class_names)

    # Calculate balanced accuracy for each epoch

    # Get true labels and predictions for the entire validation set
    y_true = []
    y_pred = []

    for i in range(len(validation_generator)):
        X_val_batch, y_val_batch = validation_generator[i]
        y_pred_batch = model.predict(X_val_batch, verbose=0)
        y_pred.extend(y_pred_batch)  # Keep the raw probabilities for precision-recall
        y_true.extend(y_val_batch)   # Keep the one-hot encoded true labels

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate balanced accuracy, confusion matrix, and F1 score
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    balanced_acc = balanced_accuracy_score(y_true_labels, y_pred_labels)
    conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

    # Print metrics
    print(f"Balanced Accuracy: {balanced_acc}")
    print(f"Accuracy on test set: {model.evaluate(validation_generator, verbose=0)[1]}")
    print(f"F1 Score: {f1_score(y_true_labels, y_pred_labels, average='weighted')}")

    # Set up a 2x2 grid for the plots
    plt.figure(figsize=(16,14), constrained_layout=True)
    
    # 1. Accuracy plot
    plt.subplot(2, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')

    # 2. Loss plot
    plt.subplot(2, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')

    # 3. Confusion matrix plot
    plt.subplot(2, 2, 3)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 4. Precision-Recall Curve plot
    plt.subplot(2, 2, 4)
    show_precision_recall(y_true, y_pred)

    # Show all the plots
    plt.show()

def show_precision_recall(y_test, predictions):
    class_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC']
    
    # Binarize the output for multiclass
    y_test = label_binarize(y_test, classes=range(len(class_names)))
    predictions = np.array(predictions)

    precision = dict()
    recall = dict()
    average_precision = dict()
    
    # Compute Precision-Recall and average precision for each class
    for i in range(len(class_names)):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], predictions[:, i])
        average_precision[i] = average_precision_score(y_test[:, i], predictions[:, i])

    # Compute micro-average Precision-Recall curve and area
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(), predictions.ravel())
    average_precision["micro"] = average_precision_score(y_test, predictions, average="micro")
    
    # Plot Precision-Recall curve
    plt.plot(recall["micro"], precision["micro"], color='gold', lw=2, label=f'Micro-average (area = {average_precision["micro"]:0.2f})')
    for i, color in enumerate(['blue', 'green', 'red', 'purple', 'brown', 'cyan', 'magenta']):
        plt.plot(recall[i], precision[i], color=color, lw=2, label=f'Class {class_names[i]} (area = {average_precision[i]:0.2f})')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve')
    plt.legend(loc='best')
